# Lab 2: Impairment Detection Agent

In underwriting, identifying medical impairments accurately is critical to assessing risk. Underwriters must scan multiple data sources—application forms, prescription histories, lab results, and medical information bureau (MIB) reports—to detect conditions like diabetes or hypertension, then cross-reference these findings against detailed underwriting guidelines to determine what factors matter for scoring.

Traditionally, this is a manual, time-intensive process prone to inconsistency. Different underwriters may interpret the same evidence differently, or miss subtle signals buried across documents.

## Why an AI Agent for Detection?

An agent solves this by:
- **Policy grounding**: Instead of relying on the model's training data, the agent retrieves authoritative guidance from the underwriting manual (the Knowledge Base you built in Lab 1), ensuring decisions align with your organization's standards.
- **Cross-document synthesis**: The agent consolidates signals from application data, Rx fills, MIB codes, and labs into a unified view, identifying impairments and the evidence supporting them—complete with page references for auditability.
- **Structured, consumable output**: The agent returns a clean JSON payload with canonical impairment IDs, scoring factors, evidence citations, and optional discrepancy flags—ready for downstream scoring in Lab 3.

## What You'll Build

In this lab, you will:
- Install dependencies and load environment variables saved in Lab 1 (`../.env`)
- Define tools (`kb_search` for Knowledge Base grounding)
- Build a Strands Impairment Detection Agent with a prompt that instructs it to scan extracted data, call the Knowledge Base for each candidate impairment, derive required scoring factors, and return structured JSON
- Run a local evaluation on sample `extracted_data` and validate the output
- Package and deploy the agent to Amazon Bedrock AgentCore Runtime for scalable, production use
- Invoke the deployed agent to confirm end-to-end detection

By the end of this lab, you'll have a deployed agent that transforms raw submission signals into normalized, policy-grounded impairment findings—the foundation for accurate, auditable risk scoring.

First, we'll need to do a bit of housekeeping. The below few cells will install the dependencies and load the environment variables saved in Lab 1 (`../.env`). 

In [1]:
# 0) Install dependencies
%pip install -qU strands-agents bedrock-agentcore bedrock-agentcore-starter-toolkit python-dotenv boto3


Note: you may need to restart the kernel to use updated packages.


In [2]:
# 1) Load environment and set up AWS clients
import os, json, logging
from pathlib import Path
from dotenv import load_dotenv
import boto3

logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Load ../.env created in Lab 1
env_path = Path('../.env')
if env_path.exists():
    load_dotenv(env_path)
    logger.info(f"Loaded env from {env_path}")
else:
    logger.warning("../.env not found. Ensure Lab 1 was completed or set env vars manually.")

REGION = os.getenv('REGION') or boto3.session.Session().region_name
BEDROCK_KB_ID = os.getenv('BEDROCK_KB_ID')
BEDROCK_KB_ARN = os.getenv('BEDROCK_KB_ARN')
FOUNDATION_MODEL_ID =  "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

session = boto3.session.Session(region_name=REGION)
kb_runtime = session.client('bedrock-agent-runtime')

print("✅ Environment loaded")
print(f"   Region: {REGION}")
print(f"   KB ID: {BEDROCK_KB_ID}")
print(f"   Model: {FOUNDATION_MODEL_ID}")


[2025-10-03 16:30:27,807] INFO - Loaded env from ../.env


✅ Environment loaded
   Region: us-east-1
   KB ID: JWWV6KBB1K
   Model: us.anthropic.claude-3-7-sonnet-20250219-v1:0


# Creating the Impairment Detection Agent 
In the next few cells, we will set up the impairment detection agent. We will use the Strands Agents framework to build it. 

### Strands Agents: what they are and how we use them here
Strands is a lightweight Python framework for building tool‑using AI agents you can run locally and deploy to Bedrock AgentCore. An agent is defined by:
- System prompt: sets the agent's role (senior underwriter) and operating rules.
- Tools (declared with `@tool`): explicit capabilities the agent is allowed to call (e.g., `kb_search` to ground in the underwriting manual).
- Model: the LLM that reasons, plans, and invokes tools to complete the task.

Why this fits impairment detection:
- Policy grounding: the agent retrieves guidance from the Knowledge Base rather than relying on recall, keeping detection aligned with the manual.
- Governed capabilities: it can only use the tools you expose, creating a clear audit surface.
- Reproducibility and deployment: the same agent you test locally packages cleanly to Bedrock AgentCore for scale, logging, and integration with your workflow/UI.

Outcome:
- The agent scans submission signals, looks up the relevant policy sections, extracts the right scoring factors and evidence with citations, and returns a clean JSON payload of impairments for downstream scoring.


Before we begin coding the agent, let's test out our Knowledge Base from Lab 1 and ensure it's working as expected. This will later be the tool we add to the impairment detection agent. 

In [14]:
# 2) Define a Knowledge Base retrieval tool
from typing import List, Dict, Any

def retrieve_kb(query: str, num_results: int = 5) -> List[Dict[str, Any]]:
    if not BEDROCK_KB_ID:
        raise RuntimeError("BEDROCK_KB_ID not set. Complete Lab 1 or set env vars.")
    resp = kb_runtime.retrieve(
        knowledgeBaseId=BEDROCK_KB_ID,
        retrievalQuery={"text": query},
        retrievalConfiguration={
            "vectorSearchConfiguration": {"numberOfResults": num_results}
        }
    )
    results = []
    for r in resp.get('retrievalResults', []):
        results.append({
            "score": r.get('score'),
            "text": r.get('content', {}).get('text', ''),
            "metadata": r.get('metadata', {})
        })
    return results

# Quick smoke test
chunks = retrieve_kb("type 2 diabetes underwriting considerations", 3)
print(f"KB returned {len(chunks)} chunks. Preview:\n- " + "\n- ".join(c['text'][:120].replace('\n',' ') for c in chunks))


KB returned 3 chunks. Preview:
- # Metabolic Conditions - Underwriting Guidelines  This section provides detailed guidelines for underwriting metabolic c
- # Endocrine Conditions - Underwriting Guidelines  This section provides detailed guidelines for underwriting endocrine c
- # Gastrointestinal & Hepatic Conditions - Underwriting Guidelines  This section provides detailed guidelines for underwr


Great! We've got a working Knowledge Base. Now, let's move on to defining our system prompt and build the impairment detection agent. 

In [4]:


SYSTEM_PROMPT = """You are a senior life insurance underwriter. Your job is to analyze the data stream for an application and identify impairments, 
scoring factors (based on the knowledge base), and evidences for those impairments. 
1. Scan the extracted datafor impairment evidence and write out an initial list of impairments.
Then for each impairment in your scratch pad, do the following:
2. Call kb_search() once and treat the markdown returned as authoritative. Make sure to record the page numbers where you found evidence for the impairment and the knowledgebase_location for the impairment to be used in the final JSON output.
3. Use the ratings tables in the returned markdown to determine a list of "scoring factors" are required to completely score that impairment and write them out. 
4. Search through the XML feeds to consolidate the values for each scoring factor, and the list of evidence for that impairment. 
5. Write out the scoring factors and evidence for that impairment.

Repeat this process for each impairment you find. Deduplicate any impairment that is found in multiple XML feeds into one listng. 

Once you have completed this process for all impairments, return a well-structured JSON response like the following:

```json  
{ 
   "impairments": [
     {
       "impairment_id": "diabetes",
       "scoring_factors": {"A1C": 8.2, "Neuropathy": true},
       "evidence": ["Rx: insulin … (Page 3,4)", "Lab: A1C 8.2 % (Page 1)"],
       "discrepancies": ["answered no to Diabetes Questionnaire but evidence of diabetes"] # optional
       "knowledgebase_location": "s3://ai-underwriting-732229910216-kb-source/manual/manual/3-medical-impairments/metabolic/type2_diabetes.md",
     
     },
     {
       "impairment_id": "hypertension",
       "scoring_factors": {"Blood Pressure": 128/92, "Age": 41, "Medication": "Lisinopril 10mg", "Duration": "At least since 2022-04-18", "Compliance": "Good - regular refills", "Target Organ Damage": "None evident", "Comorbidities": "None evident", "Family History": "Father had heart attack at age 58"},
       "evidence": ["Rx: Lisinopril 10mg for hypertension, filled 2024-01-10 (90 tablets) (Page 10)", "Rx: Lisinopril 10mg for hypertension, filled 2023-10-12 (90 tablets) (Page 11)", "MIB: Code 311C 'CARDIOVASCULAR - HYPERTENSION TREATED' from 2022-04-18 (Page 12,19)", "Application: Self-reported Lisinopril 10mg for blood pressure (Page 13)", "Application: Blood pressure reading 128/92 mmHg (Page 14)"],
       "knowledgebase_location": "s3://ai-underwriting-732229910216-kb-source/manual/manual/3-medical-impairments/metabolic/hypertension.md",
     }
   ],
   "narrative": "Agent Analysis: The applicant has a history of hypertension and diabetes. I cross checked this with the underwriting manual entries on Hypertension and Type 2 Diabetes. The hypertension is well controlled with Lisinopril 10mg, and the diabetes is well controlled with insulin. The applicant has a family history of heart attack in the father."
}
```

Explanation of the JSON output:
- impairment_id: The canonical name of the impairment.
- scoring_factors: A dictionary of scoring factors from the knowledge base entry for that impairment.
- evidence: A list of evidence for the impairment and the page numbers of the evidence.
- knowledgebase_location: The location of the knowledge base entry for the impairment (derived from the knowledgebase_location returned by the kb_search() tool).
- discrepancies: A list of discrepancies for the impairment.
- narrative: A high level summary of the analysis of all the impairments. Should include references to the knowledge base entries for the impairments that were used to generate the analysis.

   
"""

### Detection prompt: role, KB grounding, and output schema
The prompt above defines the underwriter role and a disciplined detection workflow: scan extracted data feeds for signals, retrieve the authoritative manual entry for each candidate impairment (`kb_search`), derive required scoring factors, collect evidence (with page references), and deduplicate across feeds. The agent must emit a strict JSON object:

- `impairments[]`: for each impairment, include `impairment_id`, `scoring_factors`, `evidence[]` (with page refs), optional `discrepancies[]`, and `knowledgebase_location`
- `narrative`: a concise summary tying the impairments and manual references together

This structure keeps outputs policy‑grounded, auditable, and ready for the scoring step.


In [51]:
# 3) Build a Strands Agent for impairment detection
from strands import Agent, tool
from strands.models import BedrockModel
from botocore.config import Config

@tool
def kb_search(canonical_term: str):
    print(f"[kb_search] Searching for {canonical_term}")
    """Return markdown for the top KB hit from Bedrock Knowledge Base."""
    kb_id = BEDROCK_KB_ID
    if not kb_id:
        print(f"[kb_search] Knowledge base not configured.")
        raise ValueError("Knowledge base not configured.")
    try:
        resp = kb_runtime.retrieve(
            knowledgeBaseId=kb_id,
            retrievalQuery={'text': canonical_term},
            retrievalConfiguration={'vectorSearchConfiguration': {'numberOfResults': 1}}
        )
        results = resp.get('retrievalResults') or []
        if not results:
            print(f"[kb_search] No matching documents found for {canonical_term}")
            return "No matching documents found."
        content = results[0].get('content').get('text') or {}
        location = results[0].get('location').get('s3Location').get('uri') or {}
        print(f"[kb_search] Found {canonical_term} in {location}")
        # Bedrock returns {'text': '...'} per docs
        return f"""
        knowledgebase_location: {location}
        text_content: {content}
        """
    except Exception as e:
        return f"KB retrieval error: {e}"

# Optional scratch tool mirroring lambda pattern
scratch_notes = []

@tool
def scratch_fixed(note: str) -> str:
    """Write analysis notes to a scratch pad."""
    scratch_notes.append(note)
    return "ok"

retrying_cfg = Config(
    retries={"mode": "adaptive", "max_attempts": 12}
)

MODEL_ID = FOUNDATION_MODEL_ID
model = BedrockModel(
    model_id=MODEL_ID, 
    boto_client_config=retrying_cfg)

# JSON schema aligned to the lambda's expected output (used for validation only)


agent = Agent(
    model=model,
    tools=[kb_search],
    system_prompt=SYSTEM_PROMPT
)

def run_impairment_agent(extracted_data: dict) -> dict:
    # Feed the raw JSON string directly to the agent to mirror lambda behavior
    payload = json.dumps(extracted_data, ensure_ascii=False)
    response = agent(payload)
    content = response.message.get('content', [])
    if content and isinstance(content[0], dict) and 'text' in content[0]:
        try:
            return json.loads(content[0]['text'])
        except Exception:
            # Fallback: try to extract first JSON object
            s = content[0]['text']
            import re
            m = re.search(r"\{[\s\S]*\}", s)
            return json.loads(m.group(0)) if m else {"impairments": [], "narrative": ""}
    return {"impairments": [], "narrative": ""}


In [52]:
# 4) Sample submission and local evaluation
# Replace this with the provided sample 'extracted_data' payload (merged XML feeds)
extracted_data = {
    "application": [
        {"page": 14, "blood_pressure": "128/92", "medications": ["Lisinopril 10mg"], "diabetes_questionnaire": "No"}
    ],
    "rx_history": [
        {"page": 10, "medication": "Lisinopril 10mg", "date": "2024-01-10", "qty": 90},
        {"page": 11, "medication": "Lisinopril 10mg", "date": "2023-10-12", "qty": 90},
        {"page": 3, "medication": "Insulin", "date": "2024-03-05", "qty": 30}
    ],
    "mib": [
        {"page": 12, "code": "311C", "description": "CARDIOVASCULAR - HYPERTENSION TREATED"},
        {"page": 19, "code": "311C", "description": "CARDIOVASCULAR - HYPERTENSION TREATED"}
    ],
    "labs": [
        {"page": 1, "test": "A1C", "value": 8.2, "unit": "%"}
    ]
}


### Input to the detection agent: extracted_data from prior stages
This object aggregates signals from earlier steps (e.g., application fields, Rx fills, MIB codes, lab values). In production, the detection agent consumes this payload directly from the ingestion/extraction flow. The agent consolidates the signals, consults the Knowledge Base, and returns normalized impairments with supporting evidence that downstream components can rely on.


Now we're ready to run the detection agent!

In [53]:

result = run_impairment_agent(extracted_data)
print(json.dumps(result, indent=2))


I'll analyze this application for life insurance underwriting purposes by identifying impairments, scoring factors, and evidence.

## Initial Scan for Impairments

Based on the data provided, I can identify the following potential impairments:

1. Hypertension - evidenced by:
   - Blood pressure reading of 128/92
   - Lisinopril 10mg medication
   - MIB code 311C "CARDIOVASCULAR - HYPERTENSION TREATED"

2. Diabetes - evidenced by:
   - Insulin prescription
   - A1C level of 8.2%
   - Note: The applicant answered "No" to the diabetes questionnaire (discrepancy)

Let me analyze each impairment in detail using the knowledge base.

### Impairment 1: Hypertension
Tool #1: kb_search
[kb_search] Searching for hypertension
[kb_search] Found hypertension in s3://underwriting-kb-docs-732229910216/underwriting-manual-9170309/3-medical-impairments/cardiovascular/hypertension.md
#### Scoring Factors for Hypertension:
- Blood Pressure: 128/92
- Age: Not specified in data, using the most conservative

### Interpreting the detection output
If everything is working correctly, you should see a single JSON object with an array of `impairments[]`, with fields for `impairment_id`, `scoring_factors`, `evidence`, `discrepancies`, and a `knowledgebase_location` for each item, plus a `narrative` field. 

Let's break down each of these components so you can understand the output:

**`impairments[]`** — Each detected impairment includes:

- **`impairment_id`**: Canonical name (e.g., "diabetes", "hypertension") that maps to a specific section in the underwriting manual. This ensures consistency across submissions and enables the scoring agent to look up the correct rating tables.

- **`scoring_factors`**: The specific data points required to score this impairment according to the underwriting manual's rating tables. For example, diabetes scoring may require A1C level, treatment type, duration, and complications. The detection agent extracts as many of these factors as possible from the submission data so the scoring agent (Lab 3) can calculate an accurate risk score. Missing factors may require follow-up or conservative scoring.

- **`evidence[]`**: Citations showing exactly where the impairment was detected—including the data source and page references (e.g., "Rx: insulin prescription (Page 3)", "Lab: A1C 8.2% (Page 1)"). This is critical for auditability and trust: human underwriters can verify the AI's findings, trace them back to source documents, and catch potential hallucinations or misinterpretations.

- **`knowledgebase_location`**: The S3 URI of the underwriting manual section used for this impairment (e.g., the diabetes guideline). This provides a direct link back to the policy that guided detection, enabling underwriters to review the exact criteria and rating tables the agent consulted.

- **`discrepancies[]`** (optional): Flags inconsistencies across data sources (e.g., applicant answered "No" to diabetes question but has insulin prescriptions and elevated A1C). These alerts help underwriters identify cases requiring additional investigation.

**`narrative`** — A concise, human-readable summary of the overall analysis, including detected impairments and manual references. This is useful for rendering in the Underwriting Workbench UI, giving underwriters a quick overview before diving into structured details.

**Why this structure matters:**
This JSON output is both **machine-consumable** (the scoring agent in Lab 3 ingests it directly) and **human-auditable** (underwriters can verify every finding, trace evidence, and review policy citations). It transforms raw submission signals into trustworthy, actionable intelligence.

In the next section, we'll deploy the impairment detection agent to Amazon Bedrock AgentCore Runtime, so we can integrate it into the workflow/UI. 

### Deploying to Amazon Bedrock AgentCore Runtime
Amazon Bedrock AgentCore Runtime is a secure, serverless environment for hosting and scaling AI agents. It manages scaling, session management, and security isolation, so we can focus on the agent's logic and capabilities. Deploying here gives us a dedicated, observable endpoint that integrates with the workflow/UI.

In the next cells we will:
- Generate a small runtime file with an `@entrypoint` (the callable AgentCore exposes)
- Configure the runtime using the Starter Toolkit (entrypoint, execution role, region, ECR)
- Build and deploy the agent container with CodeBuild (no local Docker required)
- Wait for the runtime endpoint to become READY
- Invoke the endpoint with a sample `submission` payload to validate end‑to‑end detection


In [54]:
agent_code = f"""
from strands import Agent, tool
from strands.models import BedrockModel
from botocore.config import Config
from bedrock_agentcore.runtime import BedrockAgentCoreApp
import os, json
import boto3
import re


# Minimal env/clients
REGION = "{REGION}"
BEDROCK_KB_ID = "{BEDROCK_KB_ID}"
FOUNDATION_MODEL_ID = "{FOUNDATION_MODEL_ID}" 
session = boto3.session.Session(region_name=REGION)
kb_runtime = session.client('bedrock-agent-runtime')

@tool
def kb_search(canonical_term: str):
    print(f"[kb_search] Searching for {{canonical_term}}")
    try:
        resp = kb_runtime.retrieve(
            knowledgeBaseId=BEDROCK_KB_ID,
            retrievalQuery={{"text": canonical_term}},
            retrievalConfiguration={{"vectorSearchConfiguration": {{"numberOfResults": 1}}}}
        )
        results = resp.get('retrievalResults') or []
        if not results:
            return "No matching documents found."
        content = results[0].get('content').get('text') or {{}}
        location = results[0].get('location').get('s3Location').get('uri') or {{}}
        print(f"[kb_search] Found {{canonical_term}} in {{location}}")
        # Bedrock returns {{'text': '...'}} per docs
        return f\"\"\"
        knowledgebase_location: {{location}}
        text_content: {{content}}
        \"\"\"
    except Exception as e:
        return f"KB retrieval error: {{e}}"

SYSTEM_PROMPT = \"\"\"
{SYSTEM_PROMPT}
\"\"\"

retrying_cfg = Config(
    retries={{"mode": "adaptive", "max_attempts": 12}}
)
model = BedrockModel(
    model_id=FOUNDATION_MODEL_ID,
    boto_client_config=retrying_cfg
)
agent = Agent(model=model, tools=[kb_search], system_prompt=SYSTEM_PROMPT)

app = BedrockAgentCoreApp()

@app.entrypoint
def strands_impairment_agent(payload: dict):
    print("payload")
    print(payload)
    raw = payload.get('submission')
    message = json.dumps(raw, ensure_ascii=False)
    res = agent(message)
    res_str = str(res)
    fence = re.search(r"```json\\s*(.*?)\\s*```", res_str, re.DOTALL)
    if fence:
        json_res = fence.group(1)
        return json.loads(json_res)
    else:
        return {{"error": "No JSON found in response"}}
    
    return json_res


if __name__ == "__main__":
    app.run()
"""



In [55]:
# agent_code is a Python str
from pathlib import Path

Path("strands_impairment_runtime.py").write_text(agent_code, encoding="utf-8")


5849

In [56]:
%%writefile requirements.txt
strands-agents
bedrock-agentcore
bedrock-agentcore-starter-toolkit
boto3
python-dotenv


Overwriting requirements.txt


### Execution role for the runtime
The runtime needs an execution role to pull container images, write logs, and invoke Bedrock resources securely. Creating it here keeps permissions explicit and least‑privileged, parameterized for your account and region, and avoids hidden console dependencies so deployment is repeatable from the notebook.


In [57]:
# Create IAM Role for AgentCore Runtime (Impairment Agent)
import json
import time
import boto3
from botocore.exceptions import ClientError

# Discover account and region
sts = boto3.client('sts')
account_id = sts.get_caller_identity().get('Account')
region_for_arns = REGION  # Reuse REGION from earlier env cell

role_name = "uw_impairment_agent_exec"
assume_role_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {"Service": ["bedrock-agentcore.amazonaws.com"]},
            "Action": "sts:AssumeRole"
        }
    ]
}

policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "ECRImageAccess",
            "Effect": "Allow",
            "Action": [
                "ecr:BatchGetImage",
                "ecr:GetDownloadUrlForLayer"
            ],
            "Resource": [
                f"arn:aws:ecr:{region_for_arns}:{account_id}:repository/*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:DescribeLogStreams",
                "logs:CreateLogGroup"
            ],
            "Resource": [
                f"arn:aws:logs:{region_for_arns}:{account_id}:log-group:/aws/bedrock-agentcore/runtimes/*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:DescribeLogGroups"
            ],
            "Resource": [
                f"arn:aws:logs:{region_for_arns}:{account_id}:log-group:*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": [
                f"arn:aws:logs:{region_for_arns}:{account_id}:log-group:/aws/bedrock-agentcore/runtimes/*:log-stream:*"
            ]
        },
        {
            "Sid": "ECRTokenAccess",
            "Effect": "Allow",
            "Action": [
                "ecr:GetAuthorizationToken"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "xray:PutTraceSegments",
                "xray:PutTelemetryRecords",
                "xray:GetSamplingRules",
                "xray:GetSamplingTargets"
            ],
            "Resource": ["*"]
        },
        {
            "Effect": "Allow",
            "Resource": "*",
            "Action": "cloudwatch:PutMetricData",
            "Condition": {
                "StringEquals": {
                    "cloudwatch:namespace": "bedrock-agentcore"
                }
            }
        },
        {
            "Sid": "BedrockAgentCoreRuntime",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:InvokeAgentRuntime"
            ],
            "Resource": [
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:runtime/*"
            ]
        },
        {
            "Sid": "BedrockAgentCoreMemoryCreateMemory",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:CreateMemory"
            ],
            "Resource": "*"
        },
        {
            "Sid": "BedrockAgentCoreMemory",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:CreateEvent",
                "bedrock-agentcore:GetEvent",
                "bedrock-agentcore:GetMemory",
                "bedrock-agentcore:GetMemoryRecord",
                "bedrock-agentcore:ListActors",
                "bedrock-agentcore:ListEvents",
                "bedrock-agentcore:ListMemoryRecords",
                "bedrock-agentcore:ListSessions",
                "bedrock-agentcore:DeleteEvent",
                "bedrock-agentcore:DeleteMemoryRecord",
                "bedrock-agentcore:RetrieveMemoryRecords"
            ],
            "Resource": [
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:memory/*"
            ]
        },
        {
            "Sid": "BedrockAgentCoreIdentityGetResourceApiKey",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetResourceApiKey"
            ],
            "Resource": [
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:token-vault/default",
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:token-vault/default/apikeycredentialprovider/*",
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:workload-identity-directory/default",
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:workload-identity-directory/default/workload-identity/uw_impairment_agent-*"
            ]
        },
        {
            "Sid": "BedrockAgentCoreIdentityGetResourceOauth2Token",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetResourceOauth2Token"
            ],
            "Resource": [
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:token-vault/default",
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:token-vault/default/oauth2credentialprovider/*",
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:workload-identity-directory/default",
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:workload-identity-directory/default/workload-identity/uw_impairment_agent-*"
            ]
        },
        {
            "Sid": "BedrockAgentCoreIdentityGetWorkloadAccessToken",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetWorkloadAccessToken",
                "bedrock-agentcore:GetWorkloadAccessTokenForJWT",
                "bedrock-agentcore:GetWorkloadAccessTokenForUserId"
            ],
            "Resource": [
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:workload-identity-directory/default",
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:workload-identity-directory/default/workload-identity/uw_impairment_agent-*"
            ]
        },
        {
            "Sid": "BedrockModelInvocation",
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel",
                "bedrock:InvokeModelWithResponseStream",
                "bedrock:ApplyGuardrail"
            ],
            "Resource": [
                "arn:aws:bedrock:*::foundation-model/*",
                f"arn:aws:bedrock:{region_for_arns}:{account_id}:*"
            ]
        },
        {
            "Sid": "RAG",
            "Effect": "Allow",
            "Action": [
                "bedrock:RetrieveAndGenerate",
                "bedrock:Retrieve"
            ],
            "Resource": "*"
        }
    ]
}

iam = boto3.client('iam')
role_arn = None
try:
    resp = iam.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy),
        Description="Execution role for Bedrock AgentCore runtime (Impairment Agent)",
        MaxSessionDuration=3600
    )
    role_arn = resp['Role']['Arn']
    # Small delay for policy propagation
    time.sleep(2)
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    else:
        raise

# Put/Update inline policy
iam.put_role_policy(
    RoleName=role_name,
    PolicyName="uw_impairment_agent_inline",
    PolicyDocument=json.dumps(policy_doc)
)

print(f"✅ IAM Role ready: {role_arn}")


✅ IAM Role ready: arn:aws:iam::732229910216:role/uw_impairment_agent_exec


In [58]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "uw_impairment_agent"
response = agentcore_runtime.configure(
    entrypoint="strands_impairment_runtime.py",
    execution_role=role_arn,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response


Entrypoint parsed: file=/Users/anhwell/projects/building-an-ai-powered-underwriting-assistant-to-enable-faster-decision-making/notebooks/lab2/strands_impairment_runtime.py, bedrock_agentcore_name=strands_impairment_runtime
[2025-10-07 10:26:13,375] INFO - Entrypoint parsed: file=/Users/anhwell/projects/building-an-ai-powered-underwriting-assistant-to-enable-faster-decision-making/notebooks/lab2/strands_impairment_runtime.py, bedrock_agentcore_name=strands_impairment_runtime
Configuring BedrockAgentCore agent: uw_impairment_agent
[2025-10-07 10:26:13,380] INFO - Configuring BedrockAgentCore agent: uw_impairment_agent
Will create new memory with mode: STM_ONLY
[2025-10-07 10:26:13,729] INFO - Will create new memory with mode: STM_ONLY
Memory configuration: Short-term memory only
[2025-10-07 10:26:13,729] INFO - Memory configuration: Short-term memory only
Found existing memory ID from previous launch: uw_impairment_agent_mem-ChbOMgDA6q
[2025-10-07 10:26:13,732] INFO - Found existing memo

ConfigureResult(config_path=PosixPath('/Users/anhwell/projects/building-an-ai-powered-underwriting-assistant-to-enable-faster-decision-making/notebooks/lab2/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/Users/anhwell/projects/building-an-ai-powered-underwriting-assistant-to-enable-faster-decision-making/notebooks/lab2/Dockerfile'), dockerignore_path=PosixPath('/Users/anhwell/projects/building-an-ai-powered-underwriting-assistant-to-enable-faster-decision-making/notebooks/lab2/.dockerignore'), runtime='Docker', region='us-east-1', account_id='732229910216', execution_role='arn:aws:iam::732229910216:role/uw_impairment_agent_exec', ecr_repository=None, auto_create_ecr=True, memory_id=None)

In [59]:
# Launch the AgentCore Runtime
launch_result = agentcore_runtime.launch()
launch_result


🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
[2025-10-07 10:26:13,754] INFO - 🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
[2025-10-07 10:26:13,755] INFO -    • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
[2025-10-07 10:26:13,755] INFO -    • No local Docker required
💡 Available deployment modes:
[2025-10-07 10:26:13,756] INFO - 💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
[2025-10-07 10:26:13,756] INFO -    • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
[2025-10-07 10:26:13,756] INFO -    • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
[2025-10-07 10:26:13,757] INFO -    • runtime.launch(local_build=True)           → Local bu

✅ Reusing existing ECR repository: 732229910216.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-uw_impairment_agent


Getting or creating CodeBuild execution role for agent: uw_impairment_agent
[2025-10-07 10:26:18,567] INFO - Getting or creating CodeBuild execution role for agent: uw_impairment_agent
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-e638e9b4a7
[2025-10-07 10:26:18,568] INFO - Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-e638e9b4a7
Reusing existing CodeBuild execution role: arn:aws:iam::732229910216:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-e638e9b4a7
[2025-10-07 10:26:18,781] INFO - Reusing existing CodeBuild execution role: arn:aws:iam::732229910216:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-e638e9b4a7
Using .dockerignore with 44 patterns
[2025-10-07 10:26:18,967] INFO - Using .dockerignore with 44 patterns
Uploaded source to S3: uw_impairment_agent/source.zip
[2025-10-07 10:26:19,144] INFO - Uploaded source to S3: uw_impairment_agent/source.zip
Updated CodeBuild project: bedrock-agentcore-uw_impairment_agent-builder
[2025-10-07 10:26:19,949] INFO - U

LaunchResult(mode='codebuild', tag='bedrock_agentcore-uw_impairment_agent:latest', env_vars=None, port=None, runtime=None, ecr_uri='732229910216.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-uw_impairment_agent', agent_id='uw_impairment_agent-ahwIDaHXAx', agent_arn='arn:aws:bedrock-agentcore:us-east-1:732229910216:runtime/uw_impairment_agent-ahwIDaHXAx', codebuild_id='bedrock-agentcore-uw_impairment_agent-builder:dab43b53-f46e-4058-92f7-e2a6c8f7f127', build_output=None)

In [60]:
# Wait for runtime to be READY
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    print(status_response)
    status = status_response.endpoint['status']
    print(status)
status


✅ MemoryManager initialized for region: us-east-1
[2025-10-07 10:27:05,427] INFO - ✅ MemoryManager initialized for region: us-east-1
🔎 Retrieving memory resource with ID: uw_impairment_agent_mem-ChbOMgDA6q...
[2025-10-07 10:27:05,787] INFO - 🔎 Retrieving memory resource with ID: uw_impairment_agent_mem-ChbOMgDA6q...
  ✅ Found memory: uw_impairment_agent_mem-ChbOMgDA6q
[2025-10-07 10:27:05,956] INFO -   ✅ Found memory: uw_impairment_agent_mem-ChbOMgDA6q
Retrieved Bedrock AgentCore status for: uw_impairment_agent
[2025-10-07 10:27:06,508] INFO - Retrieved Bedrock AgentCore status for: uw_impairment_agent


'READY'

In [62]:
# Invoke the runtime using the Starter Toolkit helper
payload = {
    "submission": {
        "application": [{"page": 14, "blood_pressure": "128/92", "medications": ["Lisinopril 10mg"], "diabetes_questionnaire": "No"}],
        "rx_history": [
            {"page": 10, "medication": "Lisinopril 10mg", "date": "2024-01-10", "qty": 90},
            {"page": 11, "medication": "Lisinopril 10mg", "date": "2023-10-12", "qty": 90},
            {"page": 3, "medication": "Insulin", "date": "2024-03-05", "qty": 30}
        ],
        "mib": [
            {"page": 12, "code": "311C", "description": "CARDIOVASCULAR - HYPERTENSION TREATED"},
            {"page": 19, "code": "311C", "description": "CARDIOVASCULAR - HYPERTENSION TREATED"}
        ],
        "labs": [
            {"page": 1, "test": "A1C", "value": 8.2, "unit": "%"}
        ]
    }
}
invoke_response = agentcore_runtime.invoke(payload)
invoke_response.get("response")[0]



'{"impairments": [{"impairment_id": "hypertension", "scoring_factors": {"Blood Pressure": "128/92 mmHg", "Medication": "Lisinopril 10mg", "Duration": "At least since 2022-04-18", "Compliance": "Good - regular refills", "Target Organ Damage": "None evident", "Comorbidities": "Diabetes"}, "evidence": ["Rx: Lisinopril 10mg for hypertension, filled 2024-01-10 (90 tablets) (Page 10)", "Rx: Lisinopril 10mg for hypertension, filled 2023-10-12 (90 tablets) (Page 11)", "MIB: Code 311C \'CARDIOVASCULAR - HYPERTENSION TREATED\' from 2022-04-18 (Page 12, 19)", "Application: Blood pressure reading 128/92 mmHg (Page 14)"], "knowledgebase_location": "s3://underwriting-kb-docs-732229910216/underwriting-manual-9170309/3-medical-impairments/cardiovascular/hypertension.md"}, {"impairment_id": "diabetes", "scoring_factors": {"A1C": 8.2, "Medication": "Insulin", "Treatment Regimen": "Insulin therapy"}, "evidence": ["Rx: Insulin prescription filled 2024-03-05 (30-day supply) (Page 3)", "Lab: A1C 8.2% (Page 

In [ ]:
# Initialize the Bedrock AgentCore client
agent_core_client = boto3.client('bedrock-agentcore')
agent_arn = launch_result.agent_arn
print(agent_arn)
# Invoke the agent
response = agent_core_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    payload=json.dumps(payload)
)
  
  
# Process and print the response
if "text/event-stream" in response.get("contentType", ""):
  
    # Handle streaming response
    content = []
    for line in response["response"].iter_lines(chunk_size=10):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    print("\nComplete response:", "\n".join(content))

elif response.get("contentType") == "application/json":
    # Handle standard JSON response
    content = []
    for chunk in response.get("response", []):
        content.append(chunk.decode('utf-8'))
    print(json.loads(''.join(content)))
  
else:
    # Print raw response for other content types
    print(response)

arn:aws:bedrock-agentcore:us-east-1:732229910216:runtime/uw_impairment_agent-ahwIDaHXAx
{'impairments': [{'impairment_id': 'hypertension', 'scoring_factors': {'Blood Pressure': '128/92', 'Medication': 'Lisinopril 10mg', 'Duration': 'At least since 2022-04-18', 'Compliance': 'Good - regular refills', 'Target Organ Damage': 'None evident', 'Comorbidities': 'Diabetes'}, 'evidence': ['Rx: Lisinopril 10mg for hypertension, filled 2024-01-10 (90 tablets) (Page 10)', 'Rx: Lisinopril 10mg for hypertension, filled 2023-10-12 (90 tablets) (Page 11)', "MIB: Code 311C 'CARDIOVASCULAR - HYPERTENSION TREATED' from 2022-04-18 (Pages 12, 19)", 'Application: Blood pressure reading 128/92 mmHg (Page 14)', 'Application: Reported medication Lisinopril 10mg (Page 14)'], 'knowledgebase_location': 's3://underwriting-kb-docs-732229910216/underwriting-manual-9170309/3-medical-impairments/cardiovascular/hypertension.md'}, {'impairment_id': 'diabetes', 'scoring_factors': {'A1C': 8.2, 'Treatment': 'Insulin'}, 'ev

### Workshop integration: update the workflow Lambda environment

If you're completing this lab as part of the full workshop, update the Step Functions–driven Lambda to point at your deployed agent.

- Open the AWS Lambda console (us‑east‑1): [AWS Lambda Console](https://console.aws.amazon.com/lambda/home?region=us-east-1#/functions)
- For this lab, update the Lambda function: `detect-impairments-lambda`
- In the function's Configuration → Environment variables, set `AGENT_ARN` to your deployed Impairment Detection Agent ARN from this notebook, then Save.

Note: Function names are defined in `static/underwriting-workshop.yaml`. 
